# Artctoc data

Cleaning GPS and analysis data

In [2]:
import pandas as pd

## Faire la jointure entre les données individus et les positions GPS

le pb est que les données attributaires des oiseaux ont une GLS ID qui ne correspond pas directement aux identifiants du fichier GPS (id)

In [3]:
df = pd.read_excel("C:\Travail\Enseignement\Cours_M2_python\Projet_Arctox\data for analyses_2010_2011_analyses.xls", 
                            sheet_name="data for analyses_2010_2011_ana")

df.columns

Index(['Year', 'Bird_ID', 'GLS_ID', 'Sex', 'Date', 'Nest', 'Nest_content',
       'Capture_method', 'Headbill', 'Culmen', 'Wing', 'Right_tarsus', 'Mass',
       'Index_body_condition', 'Muscle_Pectoral', 'Score_Personal', 'Long_Egg',
       'Long_Egg_cm', 'Larg_Egg', 'Larg_Egg_cm', 'Vol_egg', 'Arrival_date',
       'Arrival_date_num', 'Arrival_date_propre', 'Arrival_date_propre_num',
       'date_enter_nest', 'date_enter_nest_num', 'Hatch_date',
       'Hatch_date_num', 'weigh_hatching', 'Hatching_success',
       'Chick_mass_gain_(g/d)_1st_15d', 'pente_chick_growth_1-15d',
       'N_SIA_Blood', 'N_SIA_head_Feather', 'C_SIA_Blood',
       'C_SIA_head_Feather', 'Chick_sex', 'Cortico', 'Hg_HF', 'Hg_blood',
       'Season_Hg_Blood', 'Hg_BF', 'BF_side', 'Long_Median_15Oct_20Fev',
       'Lat_Median_15Oct_20Fev', 'Long_Median_DecJan', 'Lat_Median_DecJan',
       'Long_Median_Dec_20Fev', 'Lat_Median_Dec_20Fev', 'd_PL_15Oct_20Fev',
       'd_PL_DecJan', 'd_PL_1Dec_20Fev', 'Max_d_col__Dec_Jan'

In [ ]:
df.GLS_ID.unique()

In [ ]:
print(gps['id'].unique())
print(gps.shape)

### Lire le fichier complet des positions GPS

In [ ]:
import pandas as pd

df_gps = pd.read_csv("C:\Travail\Enseignement\Cours_M2_python\\2023\data\kap_hoegh_gls_complet.csv", sep=";", encoding='utf-8')
print(df_gps.shape)
gps = gpd.GeoDataFrame(
    df_gps, geometry=gpd.points_from_xy(df_gps.long, df_gps.lat), crs="EPSG:4326"
)


print(gps.columns)
print(gps.shape)
print(gps.id.unique())


## Nettoyer les positions GPS des oiseaux

Pour le nettoyage, nous allons nous inspirer d'une approche par lissage temporel, pour supprimer les latitudes anormales
- voir le package https://pypi.org/project/tsmoothie/

In [ ]:
## https://stackoverflow.com/questions/20618804/how-to-smooth-a-curve-in-the-right-way

x = np.linspace(0,2*np.pi,100) #timestampgps
y = np.sin(x) + np.random.random(100) * 0.2 #lat

# operate smoothing
import tsmoothie 
from tsmoothie.smoother import ConvolutionSmoother

smoother = ConvolutionSmoother(window_len=5, window_type='ones')
smoother.smooth(y)

# generate intervals
low, up = smoother.get_intervals('sigma_interval', n_sigma=2)

### End of adaptation

# plot the smoothed timeseries with intervals
# Avec BOKEH 
from bokeh.plotting import show, figure, output_file, output_notebook

#output_notebook() 
output_file("smoothed_data_example.html")

p = figure(width=800, height=400)

# add a line renderer for smoothed line
p.line(x, smoother.smooth_data[0], line_width =3, color='red')
p.circle(x, smoother.data[0], size =3, fill_color="white")
# add an area between low and up smoothed data
p.varea(x=x,y1=low[0], y2=up[0], alpha=0.3)

show(p)


### 1. vous avez besoin d'un axe temporel : une colonne timestamp

- https://realpython.com/python-datetime/ 
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
- https://www.delftstack.com/fr/howto/python-pandas/how-to-convert-dataframe-column-to-datetime-in-pandas/


In [ ]:
#pd.Timestamp.combine(gps.date, gps.time)
#print(gps.date+' '+gps.time)
gps['timestamp'] = gps.date+' '+gps.time
format_string = "%Y-%m-%d %H:%M:%S"
from datetime import datetime

gps.timestamp = gps.timestamp.apply(lambda x: datetime.strptime(x, format_string))


2. Mettre en oeuvre un lissage sur les latitudes
Adapt the previous code

In [ ]:
## From now, adapt the code for smoothing bad latitudes
## https://stackoverflow.com/questions/20618804/how-to-smooth-a-curve-in-the-right-way

x = gps.timestamp.values #timestampgps
y = gps.clean_lat.values #df.loc[:,['clean_lat']].values

# operate smoothing
import tsmoothie 
from tsmoothie.smoother import ConvolutionSmoother

#First simplest one : moving average of span = 5
#smoother = ConvolutionSmoother(window_len=5, window_type='ones')
#smoother.smooth(y)

#https://pypi.org/project/tsmoothie/
#https://fr.wikipedia.org/wiki/Fen%C3%AAtrage 
#Second one : moving weighted average of span = 10, using hamming function
smoother = ConvolutionSmoother(window_len=20, window_type='hamming')
smoother.smooth(y)

# generate intervals
low, up = smoother.get_intervals('sigma_interval', n_sigma=2)

### End of adaptation



In [ ]:
# plot the smoothed timeseries with intervals
from bokeh.plotting import show, figure, output_file, output_notebook

#output_notebook() 
output_file("smoothed_data.html")

p = figure(width=1600, height=800, x_axis_type='datetime')

# add a line renderer for smoothed line
p.line(x, smoother.smooth_data[0], line_width =3, color='blue')
p.circle(x, smoother.data[0], size =3, fill_color="white")
# add an area between low and up smoothed data
p.varea(x=x,y1=low[0], y2=up[0], alpha=0.3)

show(p)

In [ ]:
#Save the result
#df['smooth_lat'] = smoother.smooth_data[0]

# Joindre GPS et données d'analyse

- A gauche, les positions GPS
- A droite, les données attributaires

Pb : il faut un peu nettoyer df.GLS_ID.unique() en supprimant
- MK12-12A
- MK18-
- MK14-
- SO-
du fichier d'analyse

In [ ]:
import numpy as np
print(type(df.GLS_ID.values))
# Caster en str ce mélange de numerique et de chaînes de caractères
df.GLS_ID = df.GLS_ID.apply(lambda _: str(_))
print(type(df.GLS_ID.values))

clean_GLSID = []
for i, r in df.iterrows():
    clean_gls_id = r.GLS_ID.upper().replace('MK12-12A', '').replace('MK14-', '').replace('MK18-', '').replace('SO-', '')
    clean_GLSID = np.append(clean_GLSID, clean_gls_id)

print(pd.unique(clean_GLSID))
df['clean_GLSID'] = clean_GLSID
     

In [ ]:
## On fait la jointure sur des type identiques (id passe de int à string)
gps.id = gps.id.apply(lambda x: str(x))

In [ ]:
## Supprimer les valeurs NAN du fichier d'Analyses

#df.dropna(subset=['clean_GLSID'], inplace = True)
df  = df[df.clean_GLSID != 'NAN']
print(df.shape)
df.clean_GLSID.unique()

In [ ]:
print(df.shape)
df.clean_GLSID.unique()

In [ ]:
## Jointure

data = gps.join(df.set_index('clean_GLSID'), on='id', lsuffix='_gps', rsuffix='_ana')


print(data.shape)
print(data.columns)
data

# Poursuivre l'analyse

In [ ]:
df.Index_body_condition.dropna()